<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-tools/blob/main/notebooks/Compute_Diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Prerequisites
!pip install git+https://github.com/fair-research/bdbag
!pip install deriva
!pip install 'git+https://github.com/informatics-isi-edu/eye-ai-tools@main' --upgrade --force


In [ ]:
# login to DERIVA via Globus Auth
DEBUG_MODE = True #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'
!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}


Make a connection to the Eye AI catalog and get a reference to the main schema: eye-ai

In [ ]:
# @title Configure Grader groups
grader_list = ["Benjamin Xu", "Kendra Hong", "Kent Nguyen"] # @param
dataset_rid = "2-277M" # @param {type:"string"}
diagnosis_tag_rid = "2-35RM" # @param {type:"string"}


In [ ]:
# @title Configure Process and Diagnosis Tag
process_tag_name = "Test Diagnosis Generation" # @param {type:"string"}
Metadata = "Test Diagnosis Generation" # @param {type:"string"}
Github_URL="" # @param {type:"string"}
Process_Tag = "" # @param {type: "string"}
Description = "" # @param {type: "string"}
Github_Checksum = "" # @param {type: "string"}



Diagnosis_Tag_name = "Test Diagnosis Generation" # @param {type:"string"}
Description = "Generate new diagnosis based on a group of graders: "+str(grader_list) # @param {type:"string"}
synonyms = [] # @param


In [ ]:
from ml_utils import EyeAI
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_number = 'eye-ai')

In [ ]:
# @title Define Diagnosis Generation Rule
def CDR_func(values):
    return values.mean()

def Diag_func(values):
    # Use the mode function to get the most frequent diagnosis value
    mode_diagnosis = values.mode()

    # If there are multiple modes, take the first one
    if not mode_diagnosis.empty:
        return mode_diagnosis.iloc[0]
    else:
        # Handle the case where there is no mode (e.g., all values are unique)
        return 'Unknown'

def ImageQuality_func(values):
    mode_quality = values.mode()
    if not mode_quality.empty:
        return mode_quality.iloc[0]
    else:
        return "Unknown"

In [ ]:
# @title Extract the diagnosis in the catalog and generate new Diagnosis
all_diag = EA.image_tall(dataset_rid, diagnosis_tag_rid)
result = EA.compute_diagnosis(all_diag[all_diag["Full_Name"].isin(grader_list)],
                                 Diag_func,
                                 CDR_func,
                                 ImageQuality_func)

In [ ]:
# @title Insert New Process_Tag, Proecess, and Diagnosis_Tag
new_process_tag_RID = EA.add_term('Process_Tag', name=process_tag_name, description=grader_list, synonyms=[], exist_ok=False)
Process_RID = EA.add_process(Metadata, Github_URL, new_process_tag_RID, Description, Github_Checksum, exist_ok=False)

new_diag_tag_RID = EA.add_term('Diagnosis_Tag', Diagnosis_Tag_name, Description, exist_ok=False)


# New Section

In [ ]:
entities = EA.insert_new_diagnosis(result, new_diag_tag_RID, Process_RID)